<a href="https://colab.research.google.com/github/PrabhatGhm7/Spacy_Question_Generation/blob/main/Spacy_Question_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import spacy
nlp = spacy.load("en_core_web_sm")  #Pre trained English language

In [125]:
def generate_question(sentence):

    doc = nlp(sentence)

    # Initializes variables to store different parts of the sentence.
    subject = None
    verb =None
    obj =None
    place = None
    time = None
    reason = None
    manner = None
    aux = None  #is,was,has for gramatical correct question

    for token in doc:
        if token.dep_ =="nsubj": # identify the subject of sentence
            subject = token.text

        elif token.pos_ =="AUX":
            aux = token.text  #Getting Auxiliary verb like is, was, has, etc.

        elif token.pos_ == "VERB":
            verb = token.lemma_  # Use lemma for base form of verb

        elif token.dep_ in ["dobj", "attr"]:
            obj = token.text  # Direct object

        elif token.dep_ == "pobj" or token.ent_type_ in ["GPE","LOC", "ORG"]:
            place = token.text  # Identifies place-related words (proper nouns, locations)

        elif token.dep_ in ["prep"] and token.text in ["because","due"]:
            reason = " ".join([t.text for t in token.subtree])  # Extract reason phrase like because,due etc

        elif token.dep_ == "advmod":
            manner = token.text  # Manner (How)

        elif token.ent_type_ in ["DATE", "TIME"]:
            time = token.text  # Time expressions

    '''
    Determine the correct question verb form
    If the sentence starts with  Past tense VBD, using "did"
    If the subject is he/she/it, using does
    else, use do
    '''

    question_verb =None
    if aux:
        question_verb = aux  # Use auxiliary verbs directly like is, was etc.

    elif verb:
        root_verb = [token for token in doc if token.dep_ == "ROOT"][0]
        if root_verb.tag_ in ["VBD"]:  # Past tense use did in sentence
            question_verb = "did"

        elif subject and subject.lower() in ["he", "she", "it"]:
            question_verb = "does"
        else:
            question_verb = "do"


    # Ensure subject comes after we have auxiliary verb
    def format_question(qword, subj,main_verb):

        if question_verb and  main_verb:
            return f"{qword} {question_verb} {subj} {main_verb}?" #verb question

        elif aux and main_verb:
            return f"{qword} {aux} {subj} {main_verb}?" # is,was question

        else:
            return "Can't Generate Question"

    # Generate grammatically correct questions
    #Generating FInal question

    if subject and  verb and place:
        return format_question("Where",subject, verb)

    elif subject  and verb and time:
        return format_question("When",subject,verb)

    elif subject and  verb and obj:
        return format_question("What",subject, verb)

    elif subject and verb  and reason:
        return format_question("Why",subject, verb)

    elif subject and verb and  manner:
        return format_question("How",subject,verb)

    elif verb and obj:
        return f"Who {verb} {obj}?"

    return "Cam't Generate Sentences"



In [138]:
#trying
print(generate_question("They work in New York."))
print(generate_question("The dog sleeps on the couch."))
print(generate_question("She ate ice-cream."))
print(generate_question("Shristi speaks quickly."))
print(generate_question("The cat chased the rat."))
print(generate_question("Prabhat went to India."))
print(generate_question("The show will happen in the stadium."))

Where do They work?
Where do dog sleep?
What did She eat?
How do Shristi speak?
What did cat chase?
Where did Prabhat go?
Where will show happen?
